In [11]:
#!/usr/bin/env python
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '/home/yue/caffe-rc5/'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe
net = caffe.Net(caffe_root + 'examples/SRCNN/SRCNN.prototxt',
                       caffe_root + 'examples/SRCNN/SRCNN_iter_11112465.caffemodel', caffe.TEST)

In [12]:
def colorize(y, ycrcb):
    y[y>255] = 255
    y[y<0] = 0

    img = np.zeros((y.shape[0], y.shape[1], 3), np.uint8)
    img[:,:,0] = y
    img[:,:,1] = ycrcb[:,:,1]
    img[:,:,2] = ycrcb[:,:,2]
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    
    return img

# PSNR measure, from ANR's code
def PSNR(pred, gt):
    f = pred.astype(float)
    g = gt.astype(float)
    e = (f - g).flatten()
    n = len(e)
    rst = 10*np.log10(n/e.dot(e))
    
    return rst

In [13]:
input_stream = "./Test/videos/dance_lowres.mp4"
cap = cv2.VideoCapture(input_stream)
cur_request_id = 0
next_request_id = 1

if (cap.isOpened()== False): 
    print("Error opening video stream or file")

render_time = 0
total_time = 0
fps = 0
caffe_transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
caffe_transformer.set_transpose('data', (2, 0, 1))
caffe_transformer.set_raw_scale('data', 255.0)

In [14]:
while cap.isOpened():
        total_start = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        
        initial_w = cap.get(3)
        initial_h = cap.get(4)
        
        im_raw = cv2.resize(frame, (586, 391))
        
        im_resize = im_raw.transpose((2, 0, 1))  # Change data layout from HWC to CHW

        im_resize = cv2.resize(im_raw, (586, 391))
        ycrcb = cv2.cvtColor(im_resize, cv2.COLOR_RGB2YCR_CB)

        #print('ycrcb.shape: ', ycrcb.shape)
        im_resize = ycrcb[:,:,0]
        im_resize = im_resize.reshape((im_resize.shape[0], im_resize.shape[1], 1))
        #print('im_resize.shape: ', im_resize.shape)
        im_input = im_resize
        infer_start = time.time()
        out = net.forward_all(data=np.asarray([caffe_transformer.preprocess('data', im_input.astype(float)/255)]))
        det_time = time.time() - infer_start
        # Predict results
        mat = out['conv3'][0]

        ycrcb = ycrcb[5:-5,5:-5,:]
        im_pred = colorize(mat[0,:,:], ycrcb)
        im_pred = cv2.resize(im_pred, (im_raw.shape[1],im_raw.shape[0]))


        #################################################################################################################
        # Draw performance stats

        inf_time_message = "Inference time: {:.3f} ms".format(det_time * 1000)
        render_time_message = "OpenCV rendering time: {:.3f} ms".format(render_time * 1000)
        total_time_message = "Total Time Taken per Frame: {:.3f} ms".format(total_time * 1000)
        fps_message = "FPS: {:.1f}".format(fps);

        cv2.putText(im_pred, inf_time_message, (15, 15), cv2.FONT_HERSHEY_COMPLEX, 0.5, (200, 10, 10), 1)
        cv2.putText(im_pred, render_time_message, (15, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        cv2.putText(im_pred, total_time_message, (15, 45), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        cv2.putText(im_pred, fps_message, (15, 60), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        
        
        
        
        render_start = time.time()
        cv2.imshow("SR", im_pred)
        cv2.imshow("Original", im_raw)
        #plt.imshow(img_show)
        
        render_end = time.time()
        render_time = render_end - render_start
        total_time = render_end - total_start
        fps = float((1/total_time))
        
        key = cv2.waitKey(1)
        if key == 27:
            break
            
cv2.destroyAllWindows()

/home/yue/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/yue/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
